<div align="center" id="top">
<img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Serverless%20AI%20Workflows%20for%20Data%20%26%20ML%20Teams&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&forks=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" />

<br>
  <p>
    <a href="https://dashboard.julep.ai">
      <img src="https://img.shields.io/badge/Get_API_Key-FF5733?style=logo=" alt="Get API Key" height="28">
    </a>
    <span>&nbsp;</span>
    <a href="https://docs.julep.ai">
      <img src="https://img.shields.io/badge/Documentation-4B32C3?style=logo=gitbook&logoColor=white" alt="Documentation" height="28">
    </a>
  </p>
  <p>
   <a href="https://www.npmjs.com/package/@julep/sdk"><img src="https://img.shields.io/npm/v/%40julep%2Fsdk?style=social&amp;logo=npm&amp;link=https%3A%2F%2Fwww.npmjs.com%2Fpackage%2F%40julep%2Fsdk" alt="NPM Version" height="28"></a>
    <span>&nbsp;</span>
    <a href="https://pypi.org/project/julep"><img src="https://img.shields.io/pypi/v/julep?style=social&amp;logo=python&amp;label=PyPI&amp;link=https%3A%2F%2Fpypi.org%2Fproject%2Fjulep" alt="PyPI - Version" height="28"></a>
    <span>&nbsp;</span>
    <a href="https://hub.docker.com/u/julepai"><img src="https://img.shields.io/docker/v/julepai/agents-api?sort=semver&amp;style=social&amp;logo=docker&amp;link=https%3A%2F%2Fhub.docker.com%2Fu%2Fjulepai" alt="Docker Image Version" height="28"></a>
    <span>&nbsp;</span>
    <a href="https://choosealicense.com/licenses/apache/"><img src="https://img.shields.io/github/license/julep-ai/julep" alt="GitHub License" height="28"></a>
  </p>
  
  <h3>
    <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>
    ·
    <a href="https://x.com/julep_ai" rel="dofollow">𝕏</a>
    ·
    <a href="https://www.linkedin.com/company/julep-ai" rel="dofollow">LinkedIn</a>
  </h3>
</div>

# Hacker News Personalized Newsletter Generator

## Overview

This cookbook demonstrates how to build an intelligent newsletter generator that fetches top stories from Hacker News, personalizes content based on user preferences, and creates curated summaries. The agent analyzes story relevance using AI, scrapes full article content, and generates insightful summaries for a truly personalized reading experience.

## What This Cookbook Does

The Hacker News agent performs the following workflow:

1. **Fetches Top Stories**: Retrieves the latest top stories from Hacker News API
2. **Filters by Score**: Only includes stories that meet a minimum score threshold
3. **Scrapes Article Content**: Uses Spider web scraping to get full article content in markdown format
4. **Fetches Comments**: Retrieves top comments for each story to provide community context
5. **Personalizes Content**: Scores each story based on user's technology interests (0-100)
6. **Generates Summaries**: Creates concise, insightful summaries for relevant stories
7. **Formats Output**: Produces a clean newsletter format with titles, URLs, and summaries

## Key Features

- **Parallel Processing**: Fetches story details, article content, and comments in parallel for efficiency
- **Smart Filtering**: Two-stage filtering - first by HN score, then by personalization relevance
- **Full Content Analysis**: Goes beyond titles to analyze actual article content
- **Community Context**: Includes top comments to capture community insights
- **Customizable Parameters**: Configurable minimum score, number of stories, and user interests


In [ ]:
!pip install julep -U --quiet

In [ ]:
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

## Creating Julep Client with the API Key

Get your API key from [here](https://dashboard.julep.ai/)

In [ ]:
from julep import Client
import os

JULEP_API_KEY = "your-api-key"

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

### Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the Agent section in [Julep Concepts](https://docs.julep.ai/docs/concepts/agents).

In [ ]:
# Defining the agent
name = "Hacker News Agent"
about = "A hacker news agent that can fetch the top stories from Hacker News and summarize them."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [ ]:
import yaml

SPIDER_API_KEY = "spider-api-key"

# Defining the task
task_def = yaml.safe_load(f"""
name: HN Newsletter Generator
description: Fetch top Hacker News stories, personalize content
input_schema:
type: object
properties:
    min_score:
    type: integer
    default: 50
    num_stories:
    type: integer
    default: 10
    description: Number of stories to include in newsletter
    user_preferences:
    type: array
    items:
        type: string
    description: User's technology interests (e.g., ["AI/ML", "Python", "Startups"])

tools:
# Fetch top story IDs from Hacker News
- name: fetch_hn_stories
type: api_call
api_call:
    method: GET
    url: https://hacker-news.firebaseio.com/v0/topstories.json
    headers:
    Content-Type: application/json

# Get detailed information for a specific story
- name: get_story_details
type: api_call
api_call:
    method: GET
    url: "https://example.com"
    headers:
    Content-Type: application/json

# Fetch individual comment details
- name: get_comment_details
type: api_call
api_call:
    method: GET
    url: https://hacker-news.firebaseio.com/v0/item/{{comment_id}}.json

# Spider web scraping integration
- name: spider_fetch
type: integration
integration:
    provider: spider
    setup:
    spider_api_key: {SPIDER_API_KEY}

main:
# Step 0: Fetch top story IDs from Hacker News
# This returns an array of story IDs sorted by ranking (typically returns 500 IDs)
- tool: fetch_hn_stories
arguments:
    url: "https://hacker-news.firebaseio.com/v0/topstories.json"
label: fetch_story_ids

# Step 1: Extract first 50 story IDs
# We limit to 50 to avoid overwhelming API calls while still having enough stories to filter
- evaluate:
    story_ids: $ steps["fetch_story_ids"].output.json[:50]
    message: $ f"Fetched {{len(steps['fetch_story_ids'].output.json)}} stories, processing top 50"
label: extract_ids

# Step 2: Fetch details for each story in parallel
# Retrieves full story data (title, url, score, comments count) for each ID with parallelism of 10
- over: $ steps["extract_ids"].output["story_ids"]
parallelism: 10
map:
    tool: get_story_details
    arguments:
    method: GET
    url: $ f"https://hacker-news.firebaseio.com/v0/item/{{_}}.json"
label: all_stories

# Step 3: Extract successfully fetched story data from the API responses
- evaluate:
    stories: $ [item["json"] for item in _ if item and "json" in item]
label: extract_stories

# Step 4: Filter by score
# Only keep stories that meet the minimum score threshold (default: 50 points)
- evaluate:
    filtered: $ [s for s in steps["extract_stories"]["output"]["stories"] if "score" in s and s["score"] >= inputs.get("min_score", 50)]
label: filter_stories

# Step 5: Sort stories by score and limit to requested number
# Takes top N stories based on num_stories input parameter (default: 10)
- evaluate:
    sorted_stories: '$ steps["filter_stories"]["output"]["filtered"][:inputs.get("num_stories", 10)]'
label: sort_stories

# Step 6: Fetch full article content for each story using Spider web scraping
# Spider converts web pages to clean markdown format for easier processing
- over: $ steps["sort_stories"]["output"]["sorted_stories"]
parallelism: 4
map:
    tool: spider_fetch
    arguments:
    url: $ _['url']
    params:
        request: smart_mode
        return_format: markdown
        proxy_enabled: $ True
        filter_output_images: $ True
        filter_output_svg: $ True
        readability: $ True
        limit: 1
label: fetch_content

# Step 7: Extract scraped content from Spider responses
# Handles cases where scraping might fail for some URLs
- evaluate:
    scraped_contents: '$ [item["result"][0]["content"] if item and "result" in item and item["result"] and "content" in item["result"][0] else "" for item in _]'
label: extract_scraped_content

# Step 8: Prepare comment fetching by creating pairs of story ID and comment ID
# Gets up to 3 top comments per story for context
- evaluate:
    comment_pairs: '$ [{{"story_id": story["id"], "story_index": idx, "comment_id": kid}} for idx, story in 
enumerate(steps["sort_stories"]["output"]["sorted_stories"]) if "kids" in story for kid in story["kids"][:3]]'
label: prepare_comments

# Step 9: Fetch all comment details in parallel
# Higher parallelism (15) since comments are smaller and faster to fetch
- over: '$ steps["prepare_comments"]["output"]["comment_pairs"]'
parallelism: 15
map:
    tool: get_comment_details
    arguments:
    method: GET
    url: '$ f"https://hacker-news.firebaseio.com/v0/item/{{_[\"comment_id\"]}}.json"'
label: fetch_all_comments

# Step 10: Extract just the comment data
# Filters out any failed comment fetches
- evaluate:
    comment_results: '$ [item["json"] for item in _ if item and "json" in item and item["json"]]'
label: extract_comments

# Step 11: Group comments by their parent story index
# This maintains the relationship between comments and their stories
- evaluate:
    comments_grouped: '$ [[pair["story_index"], steps["extract_comments"]["output"]["comment_results"][i]] for i, pair in 
enumerate(steps["prepare_comments"]["output"]["comment_pairs"])]'
label: comments_with_index

# Step 12: Combine stories with their scraped content and top comments
# Creates a complete data structure for each story
- evaluate:
    stories_with_comments: '$ [dict(story, content=steps["extract_scraped_content"]["output"]["scraped_contents"][i], top_comments=[item[1] for item in 
steps["comments_with_index"]["output"]["comments_grouped"] if item[0] == i]) for i, story in enumerate(steps["sort_stories"]["output"]["sorted_stories"])]'
label: final_stories_with_comments

# Step 13: Score each story individually based on user preferences
# Uses LLM to analyze relevance to user's specified interests
# Returns a score 0-100 for personalization
- over: $ steps["final_stories_with_comments"]["output"]["stories_with_comments"]
parallelism: 10
map:
    prompt:
    - role: system
        content: |-
        $ f'''
        You are a content curator. Score this HN story's relevance to the user's interests.
        User interests: $ {{ steps[0].input.user_preferences }}

        Return only a JSON object with the relevance score (0-100).
        Return ONLY raw JSON without markdown code blocks
        '''
    - role: user
        content: >-
        $ f'''
        Story to analyze:
        Title: $ {{ _["title"] }}
        URL: $ {{ _["url"] }}
        Score: $ {{ _["score"] }}
        Content preview: $ {{ _["content"]}}
        Top comment: $ {{ _["top_comments"][0]["text"] }}

        Return format: "relevance_score" from 0 to 100
        Return only a JSON object with the relevance score (0-100).
        Return ONLY raw JSON without markdown code blocks
        '''
    unwrap: true
label: score_stories

# Step 14: Combine original stories with their relevance scores
# Creates tuples of story data and personalization score
- evaluate:
    scored_stories: '$ [{{"story": steps["final_stories_with_comments"]["output"]["stories_with_comments"][i], "relevance_score": json.loads(steps["score_stories"]["output"][i])["relevance_score"]}} for i in range(len(steps["score_stories"]["output"]))]'
label: combine_scores

# Step 15: Filter stories with relevance >= 60
# Only keep stories that are highly relevant to user's interests
- evaluate:
    personalized_stories: $ [item for item in steps["combine_scores"]["output"]["scored_stories"] if item["relevance_score"] >= 60]
label: filter_personalized

# Step 16: Generate concise summaries for each personalized story
# Creates 100-word summaries highlighting key insights
- over: $ steps["filter_personalized"]["output"]["personalized_stories"]
parallelism: 10
map:
    prompt:
    - role: system
        content: |
        Generate a concise, insightful summary (max 100 words) for this article.
        Focus on key insights and why it matters.
    - role: user
        content: >-
        $ f'''
        Title: {{ _["story"]["title"] }}
        Content: {{ _["story"]["content"] }}
        Top comments: {{ _["story"]["top_comments"] }}
        '''
    unwrap: true
label: generate_summaries

# Step 17: Prepare final output structure
# Formats all data into a clean structure
# Includes title, URLs, comment count, and generated summary
- evaluate:
    final_output: |
    $ [{{
        "title": steps["filter_personalized"]["output"]["personalized_stories"][i]["story"]["title"],
        "url": steps["filter_personalized"]["output"]["personalized_stories"][i]["story"]["url"],
        "hn_url": f"https://news.ycombinator.com/item?id={{steps['filter_personalized']['output']['personalized_stories'][i]['story']['id']}}",
        "comments_count": steps["filter_personalized"]["output"]["personalized_stories"][i]["story"].get("descendants", 0),
        "summary": steps["generate_summaries"]["output"][i]
    }} for i in range(len(steps["filter_personalized"]["output"]["personalized_stories"]))]
label: prepare_final_output

""")

In [ ]:
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [ ]:
execution = client.executions.create(
    task_id=task.id,
    input={
        "min_score": 50,
        "num_stories": 10,
        "user_preferences": ["AI/ML", "Python", "Startups", "tech"],
    }
)

### List Transitions

This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.

In [ ]:
transitions = []
of = 0

while new_transitions:= client.executions.transitions.list(execution.id, limit=100, offset=of).items:
    transitions.extend(new_transitions)
    of += 100

for i, transition in enumerate(transitions):
    print(f"Transition index: {len(transitions) - i}, type: {transition.type}, current: {transition.current}, next: {transition.next}")
    print("-" * 10)
    print(transition.output)
    print("/" * 10)

In [ ]:
execution = client.executions.get(execution.id)

print(execution.output)